# Resumen de la informacion relevante de cada estacion

El objetivo de este codigo es obtener una tabla resumen con la relacion n-n entre estaciones de calidad del aire y meteorologicas que ademas muestre cuanta informacion relevante aporta cada estacion. A partir de dicha tabla se espera decidir que estaciones aportan mas datos al estudio.

PUESTO QUE SE TRATA DE UNA GRAN CANTIDAD DE DATOS Y REALIZAR TODA LA EJECUCION LLEVARIA DEMASIADO TIEMPO, EN ESTE NOTEBOOK SE TRABAJA SOLO CON 3 ESTACIONES DE CALIDAD DEL AIRE Y EN NINGUN MOMENTO SE ESCRIBE NI LEE DE LOS ARCHIVOS (aunque el codigo se encuentra disponible en formato Markdown).

> [ESTACIONES DE CALIDAD DEL AIRE](#sitesAQ)
>
>    * [Configuracion del notebook](#config)
>    
>    * [Estaciones de españa](#espanha)
>
>         1) [Datos de las estaciones de ```saqgetr```](#estacionesSaqgetr)
>
>         2)[Datos geograficos de las estaciones](#estacionGeo)
>
>    * [Estudio de los Datos de las Estaciones](#stdio)
>
>    * [Guardar datos en csv](#saveAQ)
>
> [ESTACIONES DE LA AEMET](#sitesMto)
>
>    * [Guardar datos en csv](#saveMto)
>
> [INFORMACION RELEVANTE](#countdata)
>
>    * [Obtener los datos de Calidad del Aire](#getAQ)
>
>    * [Contar valores de Calidad del Aire](#countAQ)
>
>    * [Contar valores de Calidad del Aire](#countAQ)
>
>    * [Contar valores Meteorologicos](#countMto)
>
>    * [Agrupar todos los datos principales](#groupAll)


---
---

## ESTACIONES DE CALIDAD DEL AIRE <a id="sitesAQ"></a>

### Configuracion del notebook <a id="config"></a>

Cargamos todos los paquetes necesarios para ejecutar el notebook

In [1]:
# Loading
suppressMessages(library(saqgetr))
suppressMessages(library(tidyverse))
suppressMessages(library(lubridate))
suppressMessages(library(worldmet))
suppressMessages(library(readxl))

## Parametros de los datos <a id="param"></a>

In [3]:
# contaminantes a estudiar
pollutants <- c("no", "no2", "o3", "pm10")

# fechas de inicio y final de toma de datos
start_dt <- ymd_hms("2015-01-01 00:00:00")
end_dt <- ymd_hms("2020-10-01 00:00:00")

# fecha de inicio de confinamiento
lckdwn_strt <- ymd_hms("2020-03-14 00:00:00")

# Archivos de los datos
# NO SE TRABAJA CON ARCHIVOS
sitesAQ.fl = "a/"# "../data/csv/sitesAQ.csv"                                           
dataAQ.fl = "a/"# "../data/csv/dataAQ.csv"

sitesMto.fl = "a/"# "../data/csv/sitesMto.csv"                                         
dataMto.fl = "a/"# "../data/csv/dataMto.csv"                                          
years = 2010:2020

final.fl = "a/"# "../data/csv/nn_sites.csv"

# Guardar los datos de la calidad del aire
save.data = TRUE

### Estaciones de España <a id="espanha"></a>

#### Datos geograficos de las estaciones <a id="estacionGeo"></a>

Seleccionamos aquellas estaciones que se encuentren en un núcleo con una
población mayor o igual a cien mil habitantes `poblacion >= 100 000 hab`.

En la hoja 8 `ciudades-100000-A-JA` de la base de datos de poblaciones (`estaciones-CA.xlsx`) aparecen todas las estaciones de trafico de las ciudades de mas de 100000 habitantes con sus nombres.

In [13]:
file <- "../data/xlsx/estaciones-CA-JA.xlsx"
sheets <- c("todas", "traffic", "traffic-urban", "traffic-urban-2020",
            "traffic-suburban", "traffic-suburban-2020",
            "ciudades-100000", "ciudades-100000-A")
# xlsx files
sites.100mil <- read_excel(file, sheet=sheets[8])

In [14]:
head(sites.100mil)

Municipio,Población,Estación tráfico,Código estación,Nº estaciones Ecolog,Nº estaciones tráfico,Observaciones
<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>
A Coru<U+00F1>a,245711,CORLAB<U+00A0>1,es1138a,4,1,"Es Riazor o Santa Margarida, revisar. Datos s<U+00F3>lo hasta 2018. REVISAR"
Alcal<U+00E1> de Henares,195649,Alcal<U+00E1> de Henares,es1563a,0,1,NA
Alcobendas,117040,Alcobendas,es1564a,0,1,NA
Alicante,334887,Florida-Babel,es1915a,3,1,Urban background. Not traffic
Alicante,334888,ALACANT-EL<U+00A0>PL<U+00C1>,es1635a,3,1,Esta es de tr<U+00E1>fico
Almer<U+00ED>a,198533,MEDITERR<U+00C1>NEO,es1393a,0,1,NA


#### Datos de las estaciones de `saqgetr` <a id="estacionesSaqgetr"></a>

Importamos la informacion de las estaciones de calidad de aire de españa obtenidas de la base de datos y filtramos segun los criterios de estudio.

| Variable | Valores |
|-------|-----------|
|Contaminantes| $NO$, $NO_2$, $O_3$, $PM_{10}$|
|Fecha Inicio| 01 Enero 2015 |
|Fecha Final | 31 Diciembre 2020 |
| Site Type | traffic |
|site area | urban |

In [15]:
# obtener datos de CA de España. Salen los códigos de las estaciones
# de Calidad de aire (941)
spain.sites <- get_saq_sites() %>%
    filter(country == "spain",
           site %in% sites.100mil$"Código estación",
           site_type == "traffic",
           site_area == "urban",
           date_start <= start_dt,
           date_end >= end_dt,
           ) %>%
    select(site, site_name, latitude, longitude, elevation, 
           country, site_type, site_area, date_start, date_end)

In [16]:
# SOLO SE TRABAJA CON 3 ESTACIONES DE CALIDAD DEL AIRE
spain.sites <- spain.sites[1:3, ]

In [17]:
removed.sites <- nrow(sites.100mil) - nrow(spain.sites)

print(paste("Se han eliminado", removed.sites, "estaciones"))
print(paste("Quedan", nrow(spain.sites), "estaciones para el estudio"))

[1] "Se han eliminado 76 estaciones"
[1] "Quedan 3 estaciones para el estudio"


Agrupamos en un solo data.frame toda la informacion relevante de las estaciones de estudio

In [18]:
sites.info <- get_saq_processes() %>%
    filter(site %in% spain.sites$site,
           variable %in% pollutants,
           date_start <= start_dt,
           #date_end >= end_dt,
          ) %>%
    select(process, site, variable, variable_long, 
           period, unit, observation_count
          )

sites.geo <- sites.100mil %>%
    select("Municipio", "Población", 
           "Estación tráfico", "Código estación") %>%
    rename(site = "Código estación",)

sitesAQ <- merge(x = sites.info, y = sites.geo, by = "site", all.x = TRUE)
sitesAQ <- merge(x = sitesAQ, y = spain.sites, by = "site", all.x = TRUE)

ERROR: Error: Can't subset columns that don't exist.
[31mx[39m Column `Poblaci<U+00F3>n` doesn't exist.


In [19]:
head(sitesAQ)

ERROR: Error in head(sitesAQ): object 'sitesAQ' not found


### Guardar Datos en csv <a id="saveAQ"></a>

```R
write.csv(sitesAQ, sitesAQ.fl, row.names=FALSE)
```

---
---

## ESTACIONES DE LA AEMET <a id="sitesMto"></a>

Se pueden localizar las estaciones meteo mas cercanas a una estacion de CA mediante mapa

Obtenemos la informacion de las estaciones de calidad del aire del estudio

```R
sitesAQ <- read.csv(sitesAQ.fl)
head(sitesAQ)
```

Obtenemos la informacion de las 3 estaciones meteorologicas mas cercanas a cada estacion de calidad del aire

In [9]:
sitesMto <- data.frame()
sites.lv <- levels(as.factor(sitesAQ$site))

for (i in 1:length(sites.lv)) {
    mto <- getMeta(lat = sitesAQ[sitesAQ$site == sites.lv[i], ]$latitude[1], 
                   lon = sitesAQ[sitesAQ$site == sites.lv[i], ]$longitude[1], 
                   end.year = "current",
                   n = 3, returnMap = F)
    mto$siteAQ <- sites.lv[i]
    sitesMto <- rbind(sitesMto, mto)
}

In [10]:
head(sitesMto)

usaf,wban,station,ctry,st,call,latitude,longitude,elev(m),begin,end,code,longr,latr,dist,siteAQ
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<date>,<date>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
080250,99999,BILBAO,SP,NA,LEBB,43.301,-2.911,42.1,1973-01-01,2021-01-16,080250-99999,-0.05080653,0.7557450,5.464009,es0041a
080800,99999,VITORIA,SP,NA,LEVT,42.883,-2.724,512.7,1973-01-01,2021-01-16,080800-99999,-0.04754277,0.7484495,45.503738,es0041a
080210,99999,SANTANDER,SP,NA,LEXJ,43.427,-3.820,4.9,1973-01-01,2021-01-16,080210-99999,-0.06667158,0.7579441,73.137040,es0041a


### Guardar Datos en csv <a id="saveMto"></a>

```R
write.csv(sitesMto, sitesMto.fl, row.names=FALSE)
```

---
---

## INFORMACION RELEVANTE <a id="countdata"></a>

Calcular el numero de datos disponibles para cada estacion y almacenar en una tabla el codigo de cada estacion junto con dicho valor

### Obtener los datos de Calidad del Aire <a id="getAQ"></a>

```R
if (file.exists(sitesAQ.fl)) {
    sitesAQ <- read.csv(sitesAQ.fl, stringsAsFactors=T)
}
```

In [ ]:
if (file.exists(dataAQ.fl)) {
    dataAQ <- read.csv(dataAQ.fl)
} else {
    dataAQ <- get_saq_observations(site = levels(sitesAQ$site),
                                 variable = pollutants,
                                 valid_only = TRUE,
                                 start = start_dt,
                                 end = end_dt,
                                 verbose = TRUE
                                )

# Save data in a file for each station 
if (save.data) {
        print("Saving Data...")
        write.csv(dataAQ, dataAQ.fl, row.names=FALSE)

        split.by.site(dataAQ, site.lv="all",
                      folder="../data/csv/dataAQ/")
    }
}

In [ ]:
head(dataAQ)

### Contar valores de Calidad del Aire <a id="countAQ"></a>

In [ ]:
numCount <- c()
for (st in levels(sitesAQ$site)) {
    numCount <- c(numCount, nrow(dataAQ[dataAQ$site == st, ]))
}

nn.sitesAQ <- data.frame(siteAQ = levels(sitesAQ$site),
                         countAQ = numCount)
rm(dataAQ, numCount)

In [ ]:
head(nn.sitesAQ)

### Obtener los datos Meteorologicos <a id="getMto"></a>

```R
if (file.exists(sitesMto.fl)) {
    sitesMto <- read.csv(sitesMto.fl, stringsAsFactors=T)
}
```

In [ ]:
if (file.exists(dataMto.fl)) {
    dataMto <- read.csv(dataMto.fl)
} else {
    # Get Data from worlmet
    dataMto <- importNOAA(code = levels(sitesMto$code),
                       year = years,
                       hourly = FALSE,
                       n.cores = 4,
                       quiet = FALSE,
                       path = NA
                      )

    # Save data in a file for each station 
    if (save.data) {
        print("Saving Data...")
        write.csv(dataMto, dataMto.fl, row.names=FALSE)

        split.by.site(dataMto, site.lv="all",
                      folder="../data/csv/dataMto/")
    }
}

### Contar valores Meteorologicos <a id="countMto"></a>

In [ ]:
countMto <- NA                                                                                          
codeMto <- NA
code.lv <- levels(as.factor(dataMto$code))

for (i in 1:length(code.lv)) {
    codeMto <- rbind(codeMto, code.lv[i])
    
    countMto <- rbind(countMto, colSums(!is.na(
                                        dataMto[dataMto$code == code.lv[i],
                                        !colnames(dataMto) %in% c("date",
                                                               "station",
                                                               "latitude",
                                                               "longitude",
                                                               "elev"
                                                              )])
                                        )
                                / nrow(dataMto[dataMto$code ==code.lv[i], ])
                     )
    countMto[i+1, "code"] <- nrow(dataMto[dataMto$code == code.lv[i], ])
}
countMto <- data.frame(countMto[complete.cases(countMto), ]) %>%
                rename(countMto = code)

codeMto <- data.frame(codeMto[complete.cases(codeMto), ])
colnames(codeMto) <- c("siteMto")

countMto <- cbind(siteMto, countMto)

nn.sitesMto <- merge(x = sitesMto %>%
                        rename(siteMto=code) %>%
                        select(siteMto, dist, siteAQ),
                     y = countMto,
                     by = "siteMto", all=TRUE
                    )
rm(dataMto, countMto, sitesMto, codeMto)

---
---

## TABLA CON RELACION NN

In [ ]:
nn.sites <- merge(x = nn.sitesAQ,
                  y = nn.sitesMto,
                  by = "siteAQ", all = TRUE)
head(nn.sites)

```R
write.csv(nn.sites, final.fl, row.names=FALSE)
```